In [1]:
import os
from dotenv import load_dotenv
from typing import List

from src.KnowledgeParser import KnowledgeParser
from src.utils import save_documents_as_json, save_markdown_content, save_nodes_as_json

In [2]:
load_dotenv()
LLAMA_CLOUD_API_KEY = os.getenv("LLAMA_CLOUD_API_KEY") 
DATA_DIR = os.getenv('DATA_DIR_PATH')

In [3]:
# Initialize the KnowledgeParser
knowledge_parser = KnowledgeParser(api_key=LLAMA_CLOUD_API_KEY)

    
pdf = os.path.join(DATA_DIR, "airbnb-faq.pdf")
document = knowledge_parser.parse_pdf_sync(pdf)


Started parsing the file under job_id fcd5d454-ff2f-4dac-ad83-af5c19d17972


In [ ]:
# Save data in json and markdown files
save_documents_as_json(document, os.path.join(DATA_DIR, "documents", "markdown-entire-docs.json"))
save_markdown_content(document, os.path.join(DATA_DIR, "documents", "airbnb-faq.md")) # TODO: make this also accept a longer list of items 

In [4]:
# Let's figure out what the nodes are and look like...

nodes = knowledge_parser.node_parser.get_nodes_from_documents(document)


6it [00:00, 3208.70it/s]
100%|██████████| 6/6 [00:04<00:00,  1.22it/s]


In [7]:

first_node = nodes[0]
print(first_node.id_)
print(first_node.hash)
print(first_node.embedding)

[TextNode(id_='38d3909b-94a7-4674-b5b2-28375f628f87', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a0938633-3b59-421d-a67e-d6896f261487', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='97c43d9d824f0c722797d4732d68a2d6118d714db0a1e60cb327a1106a8cbeb2'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='id_ec006e85-ffc1-45b2-b9ac-d61edec83628_2_table_ref', node_type=<ObjectType.INDEX: '3'>, metadata={'col_schema': 'Column: Q\nType: string\nSummary: Questions related to property procedures\n\nColumn: A\nType: string\nSummary: Answers to the corresponding questions'}, hash='6bb5c4d9f914d526115124a0634d77b58f562da503635838392d9acb2c1c5278')}, text='Check-in/out Details', start_char_idx=1, end_char_idx=21, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), IndexNode(id_='id_ec006e85-ffc1-45b2-b9ac-d61ede

In [ ]:
base_nodes, objects = knowledge_parser.node_parser.get_nodes_and_objects(nodes)

save_nodes_as_json(nodes, os.path.join(DATA_DIR, "documents", "airbnb-faq-nodes.json"))
# save_documents_as_json(base_nodes, os.path.join(DATA_DIR, "documents", "markdown-entire-docs.json"))
# save_documents_as_json(objects, os.path.join(DATA_DIR, "documents", "markdown-entire-docs.json"))
# print(base_nodes[0])
# print(objects[0])